In [ ]:
import torch
from torch import nn, utils, optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from copy import deepcopy
from tqdm.auto import tqdm
import os

test_root = "./Dataset" # set this to the testing folder directory

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'GPU is available: {torch.cuda.is_available()}')
print(f'Setting default device to: {device}')

In [ ]:
print(f'Reading testing data from: {test_root} ....')

test_transform = transforms.Compose([transforms.Resize(64),
                                     transforms.CenterCrop(64),
                                     transforms.Grayscale(num_output_channels = 1),
                                     transforms.ToTensor(),
                                     lambda image: (2 * image) - 1])

test_images = datasets.ImageFolder(root = test_root, transform = test_transform)

batch_size = 1
test_data = utils.data.DataLoader(test_images , batch_size = batch_size, shuffle = False)

In [ ]:
print(f'Loading the pretrained model ....')

def ConvBlock(input_channels, output_channels):
    return nn.Sequential(
        nn.BatchNorm2d(input_channels),
        nn.LeakyReLU(0.2),
        nn.Conv2d(input_channels, output_channels, kernel_size = 3, padding = 1)
    )

def TransitionBlock(input_channels, output_channels):
    return nn.Sequential(
        nn.BatchNorm2d(input_channels),
        nn.LeakyReLU(0.2),
        nn.Conv2d(input_channels, output_channels, kernel_size = 1),
        nn.AvgPool2d(kernel_size = 2, stride = 2)
    )

class DenseBlock(nn.Module):
    def __init__(self, num_convs, input_channels, output_channels):
        super(DenseBlock, self).__init__()
        
        blocks = []
        
        for i in range(num_convs):
            blocks.append(ConvBlock((output_channels * i) + input_channels, output_channels))

        self.model = nn.Sequential(*blocks)

    def forward(self, X):
        for block in self.model:
            Y = block(X)
            X = torch.cat((X, Y), dim = 1)
        return X

output_channels, growth_rate = 64, 32
num_convs_in_dense_blocks = [4, 4, 4, 4]
blocks = []
for i, num_convs in enumerate(num_convs_in_dense_blocks):
    blocks.append(DenseBlock(num_convs, output_channels, growth_rate))
    output_channels += num_convs * growth_rate
    if i != len(num_convs_in_dense_blocks) - 1:
        blocks.append(TransitionBlock(output_channels, output_channels // 2))
        output_channels = output_channels // 2

classifier = nn.Sequential(
    nn.Conv2d(1, 64, kernel_size = 7, stride = 2, padding = 3),
    nn.BatchNorm2d(64),
    nn.LeakyReLU(0.2),
    nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1),
    *blocks,
    nn.BatchNorm2d(output_channels),
    nn.LeakyReLU(0.2),
    nn.AdaptiveMaxPool2d((1, 1)),
    nn.Flatten(),
    nn.Linear(output_channels, 10)
).to(device)

assert 'classifier.pt' in os.listdir(os.getcwd()), f'\nModel parameters are not found, make sure that the model parameters file, with name "classifier.pt" is on the same directory as the this script.\n'
classifier.load_state_dict(torch.load('./classifier.pt'))

In [ ]:
accuracy = lambda outputs, labels: (outputs.argmax(1) == labels).float().mean()

predictions = []
cum_acc = 0.0
classifier.eval()
with torch.no_grad():
    for x, y in tqdm(test_data, desc = 'Testing the Model'):
        outputs = classifier(x.cuda())
        cum_acc += accuracy(outputs, y.cuda())
        predictions += outputs.argmax(1).tolist()
accuracy = 100.0 * (cum_acc.item() / len(test_data))
print(f'\nTesting Accuracy: {accuracy}')

In [ ]:
for i, (x, y) in enumerate(test_data):
    plt.imshow(x[0][0], 'gray_r')
    plt.title(f'Prediction: {predictions[i]} | Label: {y.item()}')
    plt.axis('off')
    plt.show()